# Homework 4: Data Acquisition and Ingestion

## Task 1: API Pull

Loading environment variables below.

I've used the FMP API for the data below, with yfinance as fallback - this is because Alpha Advantage's Time series endpoint is not freely available. 

In [1]:
import os
import json
import time
import datetime
from datetime import datetime
from pathlib import Path
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sys
import yfinance

%reload_ext autoreload
%autoreload 2


config_path = Path.cwd().parent / "src"
sys.path.append(str(config_path))

from config import *
from utils import *

env_path = Path.cwd().parent / ".env"

load_env(env_path)

try:
    fmp_key = get_key("API_KEY")
    #print(f'''Loaded api key: {fmp_key}''')

except Exception as e:
    print(f'''Error retrieving key: {e} and yfinance will be used instead''')

Below pulls raw data, and saves down 1 year's worth of adjusted close prices for the given ticker. It raises a flag if date/adjclose are not part of the response of the API response

In [2]:
ticker = "AAPL"
use_api = bool(fmp_key)

end_date, start_date = year_date_interval_creator()
if use_api:

    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}"
    params = {
        "apikey": fmp_key,
        "from": end_date.strftime("%Y-%m-%d"),
        "to": start_date.strftime("%Y-%m-%d") 
    }
    r = requests.get(url, params=params)
    data = r.json()
    historical = data.get("historical", [])
    key = [k for k in historical if "adjClose" in k.keys() or "date" in k.keys()]
    assert key, f"Unexpected response keys: {list (data.keys())}"
    df_raw_api = pd.DataFrame(historical)
    df_raw_api['date'] = pd.to_datetime(df_raw_api['date'])
    df_raw_api['adjClose']=pd.to_numeric(df_raw_api['adjClose'])
    df_raw_api = df_raw_api[['date','adjClose']]

else:

    import yfinance as yf
    df_raw_api = yf.download(ticker, period="1y", interval="1d",auto_adjust=True).reset_index()[[('Date',''), ('Close',ticker)]]
    df_raw_api.columns = ['date','adjClose']
    df_raw_api['date'] = pd.to_datetime(df_raw_api['date'])
    df_raw_api['adjClose']=pd.to_numeric(df_raw_api['adjClose'])



Data pulled is validated and saved using functions from the utils module.

In [3]:
msg_dict = validate_df(df_raw_api,['date','adjClose'],{'date':'datetime64[ns]','adjClose':'float64'})
msg_dict

fname = safe_filename(prefix="api", meta={"source": "fmp" if use_api else "yfinance", "symbol": ticker})
raw_data_path = Path.cwd().parent / "data" / "raw"
out_path = raw_data_path / fname
df_raw_api.to_csv(out_path, index=False)
print("Saved:", out_path)

Saved: /Users/svolety/Desktop/bootcamp IV/bootcamp/homework/homework4and5/data/raw/api_source-fmp_symbol-AAPL_20250818-094726.csv


## Task 2: Web Scraping

Most websites with market data - yfinance/morningstar/fmp/alphaadvantage - either block requests or I need to use selenium to run the JS and pull data from the table. (otherwise it pulls a blank table) I'll return to this part of the assignment post lecture on Monday.

In [4]:
# using a wikipedia table instead

In [6]:
SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita"  # replace with permitted page
headers = {"User-Agent": "AFE-Course-Notebook/1.0 (contact: instructor@example.edu)"}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table')
    second_table = table[1]
    rows = []
    for tr in second_table.find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
        if cells:
            rows.append(cells)
    # assume first row is header
    data = rows[2:]
    df_scrape = pd.DataFrame(data, columns=['Country',*rows[1]])
except Exception as e:
    print("Scrape failed (demoing with inline HTML).", e)
    html = """
    <table>
      <tr><th>Ticker</th><th>Price</th></tr>
      <tr><td>AAA</td><td>101.2</td></tr>
      <tr><td>BBB</td><td>98.7</td></tr>
    </table>
    """
    soup = BeautifulSoup(html, 'html.parser')
    rows = []
    for tr in soup.find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
        if cells:
            rows.append(cells)
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')

msgs2 = validate_df(df_scrape, required_cols=list(df_scrape.columns), dtypes_map={})
print(msgs2)

fname2 = safe_filename(prefix="scrape", meta={"site": "example", "table": "markets"})
out_path2 = Path.cwd().parent / "data" / "raw" / fname2
df_scrape.to_csv(out_path2, index=False)
print("Saved:", out_path2)

{'na_total': 'Total NA values: 63'}
Saved: /Users/svolety/Desktop/bootcamp IV/bootcamp/homework/homework4and5/data/raw/scrape_site-example_table-markets_20250818-094816.csv
